In [0]:
from google.colab import drive
drive.mount('/content/drive')

data_dir = '/content/drive/My Drive/Computer_Vision_final/'

In [0]:
import os
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import numpy as np
import dlib 


Gather names of image files:

In [0]:
# Iterate through files in renaissance images folder and store names into array
images = []
local_download_path = data_dir + "images/"
for filename in os.listdir(local_download_path):
    tokens = filename.split(".jpg")
    name = tokens[0]
    images.append(str(name))
print(images)

In [0]:
# Iterate through files in male images folder and store names into array

train_man_images = []
local_path = data_dir + "manwomandetection/dataset/train/man/"
for filename in os.listdir(local_path):
    tokens = filename.split(".jpg")
    name = tokens[0]
    train_man_images.append(str(name))

# Iterate through files in female images folder and store names into array
train_woman_images = []
local_path = data_dir + "manwomandetection/dataset/train/woman/"
for filename in os.listdir(local_path):
    tokens = filename.split(".jpg")
    name = tokens[0]
    train_woman_images.append(str(name))

print("done")

Function used to convert output of predictor (68 points) into list of coordinates: 

In [0]:
def shape_to_np(shape, dtype='int'):
  # initialize the list of (x, y)-coordinates
	coords = np.zeros((68, 2), dtype=dtype)
 
	# loop over the 68 facial landmarks and convert them
	# to a 2-tuple of (x, y)-coordinates
	for i in range(0, 68):
		coords[i] = (shape.part(i).x, shape.part(i).y)
 
	# return the list of (x, y)-coordinates
	return coords



In [0]:
# initialize facial detector and predictor for facial keypoints 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(data_dir + 'shape_predictor_68_face_landmarks.dat')

def region_maps(srcdir, names, dstdir):
  for name in names: 
      print(name)
      im_path = data_dir + srcdir + str(name) + ".jpg"
      print(im_path)
      im = cv2.imread(im_path, cv2.IMREAD_GRAYSCALE) #read image
      h, w = im.shape

      # cv2_imshow(im)

      #run racial detector to get rectangular regions encompassing faces
      rects = detector(im, 1) 
      face_objs = []
      region_map = np.zeros((128, 128))

      #iterate through detected faces, for our data, each image only has one face so only one region will be detected.
      for (i, rect) in enumerate(rects): 
          shape = predictor(im, rect)
          shape = shape_to_np(shape)


          x = rect.left()
          y = rect.top()
          w = rect.right() - x
          h = rect.bottom() - y

          # print("size of face rectangles:", h, w)

          # region_map = np.zeros((h,w))
          region_map = np.zeros((128, 128)) # initialize region map 

          
          # To visualize where face was detected 
          cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)

          cv2.putText(im, "Face #{}".format(i + 1), (x - 10, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
          
          # for (x,y) in shape:
          #     cv2.circle(im, (x,y), 1, (0, 0, 255), -1)
              
          # separate coordinates into different regions 
          jaw = shape[0:17]
          nose = shape[27:35]
          right_eye = shape[36:42]
          right_brow = shape[17:22]
          left_eye = shape[42:48]
          left_brow = shape[22:27]
          mouth = shape[48:68]

          # f = face(jaw, nose, right_eye, right_brow, left_eye, left_brow, mouth)

          # face_objs.append(f)

          regions = [jaw, nose, right_eye, right_brow, left_eye, left_brow, mouth]  

          # rgb = [, 25, 50]

          # h, w = im.shape
          
          # iterate through regions and points in each region 
          for region in regions:
            for (x,y) in region:
              cv2.circle(im, (x,y), 3, 0, -1) # to visualize keypoints  
              x2 = x - rect.left()
              y2 = y - rect.top()
              if x2 < w and y2 < h:
                #shift keypoints to a 128x128 grid 
                x3 = int((x2 / w) * 128) 
                y3 = int((y2 / h) * 128)
                region_map[y3,x3] = 1 # add point to regoin map
      dstpath = data_dir + dstdir + str(name) + ".jpg"
      print(dstpath)
      cv2.imwrite(dstpath, region_map) # save region map 


      # cv2_imshow(region_map)
      cv2_imshow(im)




Create region maps for renaissance images

In [0]:
region_maps("images/", images, "region_maps/")

Create region maps for BAM images

In [0]:
region_maps("manwomandetection/dataset/train/man/", train_man_images, "region_maps_train/")
region_maps("manwomandetection/dataset/train/woman/", train_woman_images, "region_maps_train/") 

In [0]:
# path = data_dir + "region_maps_train/" + str(train_man_images[0]) + ".jpg" #"face_387w.jpg"
# print(path)
# img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
# cv2_imshow(img)
# img.shape
